In [133]:
import pandas as pd
import numpy as np
from datetime import datetime

## 데이터 전처리

#### 사용 Data : 규칙

Primary case : 2->1, Nan->0

장소 : 장소 index로 변경

주소 : 주소 index로 변경

신고지 : 신고지 index로 변경

직업 : 직업 index로 변경

성별 : 내/외국인 분리하여 is_korean feature 사용
       남자 1, 여자 0

집단발병일, 확진일 : 차이를 계산하여 diff_days column으로 사용

방역 수칙 sum : Nan -> 같은 클러스터 값으로 대치 

위험도 sum : Nan -> 같은 클러스터 값으로 대치

CT_R : Nan -> 전체 평균

CT_E : Nan -> 전체 평균

연령 : 장소 index 별 연령 평균으로 결측값 대치



In [134]:
df = pd.read_csv("집단사례보고.csv" )
cluster_df = df["장소"]
my_list = [s for s in cluster_df.unique() if not s.endswith(')')]

### 장소 mapping

In [135]:
mapping={'병원':12,
 '요양시설':11,
 '어린이집':18,
 '초등학교':9,
 '직장':7,
 '병원콜센터':6,
 '요양병원':12,
 '장애인거주시설':11,
 '물류센터':3,
 '자동차전시장':4,
 '사우나':8,
 '태권도장':14,
 '헬스장':14,
 '시청':8,
 '교회':13,
 '컨설팅업':7,
 '음료제조업':19,
 '다이어트장':14,
 '동호회':17,
 '대학교':9,
 '음식점':15,
 '학원':10,
 '학교':9,
 '광고물제조업':1,
 '세무사사무소':6,
 '당구장':14,
 '화장품제조업':1,
 '자동차연구소':6,
 '콜센터':6,
 '골판지제조업':5,
 '실내운동시설':14,
 '공장':1,
 '제조업':1,
 '불교':13,
 '단란주점':16,
 '연구기관':6,
 '자영업':15,
 '가구점':4,
 '이슬람사원':13,
 '농산물유통센터':4,
 '농산물제조판매업':19,
 '건설현장':2,
 '보험사':6,
 '축산업':19,
 '유흥주점':16,
 '장례식장':8,
 '건설회사':2,
 '생산공장':1,
 '농수산물류센터':4,
 '계란공장':19,
 '의료기매장':6,
 '지인모임':17,
 '유치원':18,
 '건축사무소':6,
 '학교급식실':9,
 '성당':13,
 '체대입시학원':14,
 '댄스학원':14,
 '어학원':10,
 '탁구장':14,
 '기숙학원':8,
 '고등학교':9,
 '특수학교':9,
 '보험회사':7,
 '반도체제조업':1,
 '보도방':16,
 '외국인숙소':8,
 '피아노학원':10,
 '보드게임장':17,
 '중학교':9,
 '의류공장':1,
 '보습학원':10,
 '음악학원':10,
 'KT본사':7,
 '제조업체생산라인':1,
 '신발제작공장':1,
 '제조업체':1,
 '화물운송협회':3,
 '자동차부품생산업체':1,
 '제조업공장':1,
 '자동차공장':1,
 '직업기술학원':10,
 '가공식품도매업':4,
 '제조업공장사내기숙사':8,
 '주점':16,
 '식료품제조업체':19,
 '무속신앙':13,
 '통신전선설치업체':6,
 '개인과외':17,
 '교육원':8,
 '사내기숙사':8,
 '노래방':16,
 '전자회사':7,
 '주짓수':14,
 '무에타이':14,
 '고시원':8,
 '식품제조업체':19,
 '도금업체':1,
 '가전제조판매업체':4,
 '산업체':1,
 '시멘트제조업체':1,
 '아이스링크장':14,
 '아파트건설현장':2,
 '고시텔':8,
 '대안학교':9,
 '자동차부품도장업체':6,
 '파이프가공공장':1,
 '주간보호센터':11,
 '공공기관':8,
 '육류가공업체':14,
 '렌탈업체':6,
 '대학교기숙사':8,
 'IT업':7,
 '플라스틱제조업':1,
 '사진스튜디오':6,
 '가구제조업':1,
 '전자부품제조업':1,
 '기계제조업':1,
 '자동차부품제조업':1,
 '장애인복지시설':11,
 '박스제조업':1,
 '장비제조업':1,
 '장비제조업(지인':1,
 '외국인근로자기숙사':8,
 '도장및피막처리업':1,
 '전자전기제품제조업':1,
 '인력사무소':6,
 '반도체장비부품가공업':1,
 '염료제조업':1,
 '건축제조업':1,
 '중공업':2,
 '알루미늄주조업':1,
 '축산물가공업':4,
 '모델하우스':6,
 '사료제조업':19,
 '인쇄업체':6,
 '영어학원':10,
 '마스크제조업':1,
 '콘크리트제조업':1,
 '금속처리업':1,
 '자동차정비업':6,
 '재활용품분류센터':5,
 '육류가공제조업':19,
 '농구단':14,
 '금속가공제품제조업':1,
 '냉장창고업':19,
 '여과기제조업':1,
 '식품제조업':19,
 '금형제조업':1,
 '코인노래방':16,
 '게임회사':7,
 '재활용업체':5,
 '원단제조업':1,
 '철강제조업':1,
 '고속도로사업소':6,
 '대학교체육부':14,
 '광학부품전문제조업':1,
 '건설업':2,
 '섬유가공업':1,
 '택배물류센터':4,
 '플라스틱용기제조업':1,
 '대형마트':4,
 '요양원':11,
 '골판지상자제조업':1,
 '식품가공업':19,
 '분양상담소':6,
 '농공시설업':1,
 '포장용상자제조업':1,
 'PC방':16,
 '염색가공업':4,
 '학교축구부':14,
 '대형마트물류센터':4,
 '섬유제조업':1,
 '양돈업':19,
 '전자상거래소매업':6,
 '종이제조업':1,
 '플라스틱재활용페기업체':5,
 '피혁가공판매업':5,
 '택배영업소':4,
 '간판제조업':1,
 '자동차판매업':4,
 '무역업':4,
 '피혁원단제조업':1,
 '대부중개업':6,
 '인쇄업':6,
 '태권도학원':10,
 '전자제품제조업':1,
 '의류선별및수출업':4,
 '창호제조업':1,
 '낙농제품제조업':19,
 '종교시설':13,
 '회사기숙사':8,
 '제조공장':1,
 '교욱원':10,
 '렌탈업쳬':6,
 '플라스틱재활용폐기업체':5
 }

In [136]:
def remove_bracket(s):
    """
    문자열에서 괄호안의 문자를 제거하는 함수
    """
    idx = s.find('(') # 괄호 시작 위치 찾기
    if idx != -1:
        s = s[:idx] # 괄호 이전 문자열만 남기기
    return s

def replace_column(df, column_name):
    """
    데이터프레임의 특정 컬럼에서 괄호안의 문자를 제거하고, 괄호 밖의 문자만 남긴 새로운 컬럼을 생성하는 함수
    """
    new_column_name = column_name + '_index'
    df[new_column_name] = df[column_name].apply(remove_bracket)
    df[new_column_name] = df[new_column_name].str.replace('(', '').str.replace(')', '') # 괄호 제거
    return df

In [137]:
df = replace_column(df, '장소')
df["장소"] = df["장소_index"].map(mapping)
df["장소클러스터"]=df["장소"]
df["장소"]=df["장소_index"]
df.drop("장소_index", axis=1, inplace=True)


In [138]:
def clean_value(value):
    if value =='강원시영월군':
        value = '영월군'
    if '시' in value and value != '시흥시':
        value = value.split('시')[0] + '시'
    if '연천' in value:
        value = '연천시'
    if '경기도' in value:
        value = value.replace('경기도 ', '')
    if '경기도' in value:
        value = value.replace('경기도', '')
    if '고양ㅅ' in value  or value == "덕양구" or value =='고양일산서구' or value =='고양덕양':
        value = '고양시'
    if '성남수분당구' in value:
        value = value.replace('성남수분당구', '성남시')
    if '안사시' in value:
        value = value.replace('안사시', '안산시')
    if value == '성북구'or value ==  '금천구'or value ==  '동대문구'or value ==  '구로구'or value ==  '동작구' or value ==  '영등포구' or value =="관악구" or value =="도봉구" or value =="용산구":
        value = '서울시'
    if value == "계양구" or value == '인천부평':
        value="인천시"
    if value == "의정뷧":
        value="의정부시"
    if value == "동두천":
        value="동두천시"
    if value == "양평":
        value="양평군"
    if value == "충북진천":
        value="진천군"
    if value == "충남부여":
        value="부여군"
    if value == "강원도철원군":
        value="철원군"
    if value == "경남고성군":
        value="고성군"
    if value == "충남당진":
        value="당진시"
    if value == "전북전주":
        value="전주시"
    if value == "충남태안":
        value="태안군"
    if value == "충북충주":
        value="충주시"
    if value == "강원춘천":
        value="춘천시"
    if value == "포천":
        value="포천시"
    if value == "경북고창":
        value="고창군"
    return value

In [139]:
df['주소'] = df['주소'].apply(clean_value)

df["주소"].unique()

array(['광주시', '부천시', '안양시', '용인시', '파주시', '고양시', '평택시', '의정부시', '양평시',
       '성남시', '화성시', '남양주시', '하남시', '수원시', '안산시', '과천시', '구리시', '포천시',
       '광명시', '김포시', '오산시', '군포시', '안성시', '여주시', '시흥시', '의왕시', '이천시',
       '가평군', '동두천시', '연천시', '양주시'], dtype=object)

In [140]:
add_map={'광주시':0, '부천시':1, '안양시':2, '용인시':3, '파주시':4, '고양시':5, '평택시':6, '의정부시':7, '양평시':8,
       '성남시':9, '화성시':10, '남양주시':11, '하남시':12, '수원시':13, '안산시':14, '과천시':15, '구리시':16, '포천시':17,
       '광명시':18, '김포시':19, '오산시':20, '군포시':21, '안성시':22, '여주시':23, '시흥시':24, '의왕시':25, '이천시':26,
       '가평군':27, '동두천시':28, '연천군':29, '양주시':30}

In [141]:
df['주소_index']=df['주소'].map(add_map)

### 신고지 mapping

경기도를 제외한 지역의 세분화는 최소화 하였음

In [142]:
def clean_value_rep(value):
    if value =='강원시영월군':
        value = '영월군'
    if '시' in value and value != '시흥시':
        value = value.split('시')[0] + '시'
    if '연천시' in value:
        value = '연천군'
    if '경기도' in value:
        value = value.replace('경기도 ', '')
    if '경기도' in value:
        value = value.replace('경기도', '')
    if '고양ㅅ' in value  or value == "덕양구" or value =='고양일산서구' or value =='고양덕양':
        value = '고양시'
    if '성남수분당구' in value:
        value = value.replace('성남수분당구', '성남시')
    if value == '구로시'or value == '송파구'or value == '노원시':
        value = "서울시"
    if value =="부평시":
        value = '인천시'
    if value =="일산시":
        value = '고양시'
    if value == "대구시" or value == "천안시" or value == '충북시'or value == '전주시'or value == '춘천시'or value == '태안군'or value == '포항시'or value == '전남시'or value == '괴산시'or value == '울산시'or value == '.'or value == '충주시'or value == '청주시'or value == '충남시'or value == '전북고창'or value == '영월군'or value == '어루시'or value == '김천시'or value == '당진시'or value == '부산시'or value == '원주시'or value == '진천군'or value == '부여군'or value == '대전시'or value == '세종시'or value == '철원군'or value == '고성군':
        value="전국"
    return value

In [143]:
df['신고지']=df['신고지'].apply(clean_value_rep)
df['신고지'].unique()

array(['광주시', '부천시', '안양시', '용인시', '파주시', '서울시', '고양시', '평택시', '김포시',
       '양주시', '의정부시', '양평군', '성남시', '화성시', '남양주시', '수원시', '인천시', '전국',
       '하남시', '안산시', '과천시', '군포시', '구리시', '포천시', '광명시', '시흥시', '오산시',
       '의왕시', '안성시', '여주시', '이천시', '양평시', '가평군', '강원춘천', '충남태안', '인천부평',
       '전북전주', '동두천시', '연천군', '성북구', '계양구', '금천구', '동대문구', '구로구', '동작구',
       '영등포구', '연천', '포천', '동두천', '도봉구', '의정뷧', '양평', '관악구', '용산구', '안사시',
       '충남당진', '충북충주', '충북진천', '충남부여', '강원도철원군', '경남고성군'], dtype=object)

In [144]:
rep_map={'광주시':0, '부천시':1, '안양시':2, '용인시':3, '파주시':4, '고양시':5, '평택시':6, '의정부시':7, '양평시':8,
       '성남시':9, '화성시':10, '남양주시':11, '하남시':12, '수원시':13, '안산시':14, '과천시':15, '구리시':16, '포천시':17,
       '광명시':18, '김포시':19, '오산시':20, '군포시':21, '안성시':22, '여주시':23, '시흥시':24, '의왕시':25, '이천시':26,
       '가평군':27, '동두천시':28, '연천군':29, '양주시':30, '양평군':8, '인천시':31, "서울시":32, "전국":33}

In [145]:
df['신고지_index']=df['신고지'].map(rep_map)

In [146]:
df.drop(['증상발현일','시리얼','확진자번호','장소','상호','주소','신고지','직업명','CT_N','감염경로종합평가','개인위험특성','감염장소위험특성','백신접종사항','주요감염원인','클러스터외확진자','type_f','infector_id','primary_case_id','접촉자','Secon_attack','SA_F_Incld','방역수칙_1','방역수칙_2','방역수칙_3','방역수칙_4','방역수칙_5','방역수칙_행정명령','위험도_1','위험도_2','위험도_3','위험도_4(군집도)','위험도_5','위험도_6','기타'],inplace=True,axis=1)
df

,클러스터,집단발병일,성별,연령,직업대분류,확진일,CT_R,CT_E,primary case,확진자,방역수칙_sum,sum,장소클러스터,주소_index,신고지_index
0,1,20201017,2,.,4,20201016,15.92,14.49,2,47,1,8,12,0.0,0.0
1,2,20201020,1,.,0,20201018,31.81,31.74,2,4,3,7,11,1.0,1.0
2,2,20201020,2,.,0,20201020,29.05,28.12,0,4,3,7,11,1.0,1.0
3,2,20201020,2,.,0,20201020,14.28,12.64,0,4,3,7,11,1.0,1.0
4,2,20201020,2,.,0,20201020,22.77,21.44,0,4,3,7,11,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5896,448,20211102,2,2,0,20211031,17.11,15.41,0,10,4,8,18,10.0,10.0
5897,448,20211102,2,33,2,20211031,20.51,18.7,0,10,4,8,18,10.0,10.0
5898,448,20211102,2,2,0,20211011,30.31,28.65,0,10,4,8,18,10.0,10.0
5899,448,20211102,2,32,99,20211030,15.61,14.09,0,10,.,.,18,10.0,10.0


### 기타 항목 전처리


In [147]:
# 성별 1, 2 한국 남자, 한국 여자와 3, 4 외국남자 외국여자를 is_koren과 성별 두개의 coulumn으로 분리

df.loc[(df['성별'].isin(['1', '2'])), 'is_korean'] = '1'
df.loc[(df['성별'].isin(['3', '4'])), 'is_korean'] = '0'
df.loc[(df['성별'] == '3'), '성별'] = '1'
df.loc[(df['성별'] == '4'), '성별'] = '2'

In [148]:
# primary case는 1로 통일

df['primary case'].replace('2', '1', inplace=True)

In [149]:
# 결측치 대체
df['CT_R'].replace('.', np.nan, inplace=True)
df['CT_R'] = df['CT_R'].astype(float)
df['CT_R'].fillna(df['CT_R'].mean(), inplace=True)

df['CT_E'].replace('.', np.nan, inplace=True)
df['CT_E'] = df['CT_E'].astype(float)
df['CT_E'].fillna(df['CT_E'].mean(), inplace=True)

# 각각의 평균값으로 결측치 사용
df["CT_R"] = df["CT_R"].fillna(df["CT_R"].mean())
df["CT_E"] = df["CT_E"].fillna(df["CT_E"].mean())

In [150]:
# 장소에 해당하는 연령의 평균으로 결측치 사용
df['연령'].replace('.', np.nan, inplace=True)
plc_idx=[]
for i in range(1,20):
    x=df[df["장소클러스터"]==i]
    try:
        plc_idx.append(x["연령"].astype("float").mean(skipna=True).round().astype("int32"))
        print(i,x["연령"].astype("float").mean(skipna=True).round().astype("int32"))
    except:
        print(i)
        pass

for i in range(1, 20):
    cluster_mean = plc_idx[i-1]  # plc_idx 리스트에서 해당 장소클러스터에 대한 연령 평균값 가져오기
    df.loc[df['장소클러스터'] == i, '연령'] = df.loc[df['장소클러스터'] == i, '연령'].fillna(cluster_mean)


1 39
2 43
3 35
4 40
5 43
6 40
7 42
8 38
9 22
10 19
11 68
12 64
13 44
14 32
15 45
16 30
17 44
18 21
19 42


### 집단 발병일 과 확진일의 차이를 구해 diff_days로 계산, 클러스터의 지속 기간으로 간주하였음.

다만, 형식이 올바르지 못한 부분에 대해서는 수작업으로 이를 처리하여주었음

cluster 8 : 오류데이터? 제거 필요 

cluster 9, 80, 255 : 집단발병일 날짜 형식 어김. 확진일과 비교 임의로 수정.

cluster 105 확진자번호 946 : 확진 날짜 오타 추정 2010611 → 20210611

cluster 112 확진자번호 997 : 확진 날짜 오타 추정 2010614→20210614 

cluster 139 확진자번호 1408 : 확진 날짜 오타 추정 20210708→202210708

cluster 141 확진자번호 1422 : 확진 날짜 오타 추정 20211214→20201214

cluster 171 : 오류데이터? 제거 필요

cluster 220 확진자번호 2397 : 확진 날짜 오타 추정 20210617→20210817 (primary보다 이후에 감염되어야 하므로)

cluster 229 확진자번호 2514 : 확진 날짜 오타 추정 20210406→20210106

cluster 270 확진자번호 3129 : 확진 날짜 오타 추정 20201823→20210823

cluster 272 확진자번호 3153 : 확진 날짜 오타 추정 20201824→20210824

cluster 358 확진자번호 4638 : 확진 날짜 오타 추정 2210930→20210930

cluster 359 확진자번호 4394 : 확진 날짜 오타 추정 20011001→20211001  

cluster 373 확진자번호 4619 : 확진 날짜 오타 추정 2021005→20211005

cluster 375 확진자번호 4652 : 확진 날짜 오타 추정 20011005→20211005

cluster 378 확진자번호 4686 : 확진 날짜 오타 추정 2021006→20211006 

cluster 388 확진자번호 4877 : 확진 날짜 오타 추정 2021009→20211009 

cluster 431 확진자번호 5561 : 확진 날짜 오타 추정 20221028 → 20211028

cluster 445 확진자번호 5852 : 확진 날짜 오타 추정 2211030 → 20211030


In [151]:
for idx, row in df.iterrows():
    try:
        df.loc[idx, '집단발병일'] = pd.to_datetime(row['집단발병일'], format='%Y%m%d')
    except ValueError:
        print(f"올바르지 않은 형식의 날짜입니다: {row['집단발병일']}, {idx+1}번째 행")

    try:
        df.loc[idx, '확진일'] = pd.to_datetime(row['확진일'], format='%Y%m%d')
    except ValueError:
        print(f"올바르지 않은 형식의 날짜입니다: {row['확진일']}, {idx+1}번째 행")

# 날짜 차이 계산
# df['diff_days'] = (df['확진일'] - df['집단발병일']).dt.days

올바르지 않은 형식의 날짜입니다: 20201232, 67번째 행
올바르지 않은 형식의 날짜입니다: 20201233, 68번째 행
올바르지 않은 형식의 날짜입니다: 20201234, 69번째 행
올바르지 않은 형식의 날짜입니다: 20201235, 70번째 행
올바르지 않은 형식의 날짜입니다: 20201236, 71번째 행
올바르지 않은 형식의 날짜입니다: 20201237, 72번째 행
올바르지 않은 형식의 날짜입니다: 20201238, 73번째 행
올바르지 않은 형식의 날짜입니다: 20201239, 74번째 행
올바르지 않은 형식의 날짜입니다: 20201240, 75번째 행
올바르지 않은 형식의 날짜입니다: 20201241, 76번째 행
올바르지 않은 형식의 날짜입니다: 20201242, 77번째 행
올바르지 않은 형식의 날짜입니다: 20201243, 78번째 행
올바르지 않은 형식의 날짜입니다: 20201244, 79번째 행
올바르지 않은 형식의 날짜입니다: 20201245, 80번째 행
올바르지 않은 형식의 날짜입니다: 20201246, 81번째 행
올바르지 않은 형식의 날짜입니다: 20201247, 82번째 행
올바르지 않은 형식의 날짜입니다: 20201248, 83번째 행
올바르지 않은 형식의 날짜입니다: 20201249, 84번째 행
올바르지 않은 형식의 날짜입니다: 20201250, 85번째 행
올바르지 않은 형식의 날짜입니다: 20201251, 86번째 행
올바르지 않은 형식의 날짜입니다: 20201252, 87번째 행
올바르지 않은 형식의 날짜입니다: 20201253, 88번째 행
올바르지 않은 형식의 날짜입니다: 20201254, 89번째 행
올바르지 않은 형식의 날짜입니다: 20201255, 90번째 행
올바르지 않은 형식의 날짜입니다: 20201256, 91번째 행
올바르지 않은 형식의 날짜입니다: 20201257, 92번째 행
올바르지 않은 형식의 날짜입니다: 20201258, 93번째 행
올바르지 않은 형식의 날짜입니다: 20201259,

올바르지 않은 형식의 날짜입니다: 20210832, 2924번째 행
올바르지 않은 형식의 날짜입니다: 20210833, 2925번째 행
올바르지 않은 형식의 날짜입니다: 20210834, 2926번째 행
올바르지 않은 형식의 날짜입니다: 20210835, 2927번째 행
올바르지 않은 형식의 날짜입니다: 20210836, 2928번째 행
올바르지 않은 형식의 날짜입니다: 20210837, 2929번째 행
올바르지 않은 형식의 날짜입니다: 20210838, 2930번째 행


### 위 같은 절차를 거쳐 1차적인 전처리를 통해 ```data_final_mod.csv``` 생성

### cutoff date 별로 전처리 및 각각 csv 파일 생성

In [152]:
data = pd.read_csv("data_final_mod.csv",encoding='euc-kr' )
data=data[['cluster', 'age', 'CT_R', 'CT_E', 'dis', 'danger','gender', 'is_korean', 'primary case', 
                   'job_idx', 'rep_idx', 'place_idx', 'add_idx', 'diff_days','y']]
data = data.sort_values(by=['cluster', 'diff_days'], ascending=[True, True])
## cluster 별로 상이한 diff days 정규화
min_diff_days = data.groupby('cluster')['diff_days'].min()
data['diff_days'] = data.apply(lambda row: row['diff_days'] - min_diff_days[row['cluster']], axis=1)

## 정답 label d (cluster duration) 생성
grouped = data.groupby('cluster')['diff_days'].max()
data['d'] = data.apply(lambda row: grouped[row['cluster']] - row['diff_days'], axis=1)

## 정답 label y (cluster patients) 생성, 현재 일자를 제외한 미래의 환자 수 count
for _, group_data in data.groupby('cluster'):
    for index, row in group_data.iterrows():
        group_data.loc[index, 'y'] = group_data[group_data['diff_days'] > row['diff_days']].shape[0]
    data.update(group_data)

## cutoff 기준 생성
data['cut_date'] = data['cluster'].map(lambda x: grouped[x] + 1)
data.to_csv("data_task.csv",mode="w",encoding='euc-kr',index=False)
data

,cluster,age,CT_R,CT_E,dis,danger,gender,is_korean,primary case,job_idx,rep_idx,place_idx,add_idx,diff_days,y,d,cut_date
0,1,64,15.92,14.49,1,8,0,1,1,4,0,12,0,0.0,0,0.0,1.0
2,2,68,29.05,28.12,3,7,0,1,0,0,1,11,1,0.0,1,2.0,3.0
3,2,68,14.28,12.64,3,7,0,1,0,0,1,11,1,0.0,1,2.0,3.0
4,2,68,22.77,21.44,3,7,0,1,0,0,1,11,1,0.0,1,2.0,3.0
5,2,68,28.46,27.67,3,7,0,1,0,4,1,11,1,0.0,1,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5886,448,2,17.11,15.41,4,8,0,1,0,0,10,18,10,0.0,2,20.0,21.0
5887,448,33,20.51,18.70,4,8,0,1,0,2,10,18,10,0.0,2,20.0,21.0
5890,448,33,16.77,15.43,4,8,0,1,0,99,10,18,10,0.0,2,20.0,21.0
5889,448,32,15.61,14.09,4,8,0,1,0,99,10,18,10,1.0,1,19.0,21.0


In [11]:
import pandas as pd
# data = pd.read_csv("data_task.csv",encoding='euc-kr' )
## cutoff data csv 1 ~ 5 까지 생성
for cut_off in range(1,6):
    data = pd.read_csv("data_task.csv",encoding='euc-kr' )
    data.drop(data[(data['cut_date'] < cut_off) | (data['diff_days'] >= cut_off)].index, inplace=True)
    data['cluster'] = data['cluster'].map({value: idx for idx, value in enumerate(sorted(data['cluster'].unique()))})
    data['cut_date'] = cut_off
    data.to_csv(f"data_cut_{cut_off}.csv",mode="w",encoding='euc-kr',index=False)

1
2
3
4
5


In [154]:
import pandas as pd

file_list = ['data_cut_1.csv', 'data_cut_2.csv', 'data_cut_3.csv', 'data_cut_4.csv', 'data_cut_5.csv']
combined_df = pd.DataFrame()

# 파일 순회 및 데이터프레임 합치기
for file in file_list:
    df = pd.read_csv(file)
    if len(combined_df) != 0:   
        print(combined_df['cluster'].iloc[-1])
        df['cluster']=df['cluster']+combined_df['cluster'].iloc[-1]+1
    combined_df = pd.concat([combined_df, df], ignore_index=True)
print(combined_df['cluster'].unique())
combined_df.to_csv(f"data_cut_0.csv",mode="w",encoding='euc-kr',index=False)
combined_df

444
444
444
444
[0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51
 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99
 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153
 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171
 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189
 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207
 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225
 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243
 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261
 262 263 264 265 266 267 268 269 

,cluster,age,CT_R,CT_E,dis,danger,gender,is_korean,primary case,job_idx,rep_idx,place_idx,add_idx,diff_days,y,d,cut_date
0,0,64,15.92,14.49,1,8,0,1,1,4,0,12,0,0.0,0,0.0,1
1,1,68,29.05,28.12,3,7,0,1,0,0,1,11,1,0.0,1,2.0,1
2,1,68,14.28,12.64,3,7,0,1,0,0,1,11,1,0.0,1,2.0,1
3,1,68,22.77,21.44,3,7,0,1,0,0,1,11,1,0.0,1,2.0,1
4,1,68,28.46,27.67,3,7,0,1,0,4,1,11,1,0.0,1,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,444,32,27.38,20.50,4,8,0,1,0,2,10,18,10,0.0,2,20.0,1
1937,444,44,17.03,15.50,4,8,0,1,0,2,10,18,10,0.0,2,20.0,1
1938,444,2,17.11,15.41,4,8,0,1,0,0,10,18,10,0.0,2,20.0,1
1939,444,33,20.51,18.70,4,8,0,1,0,2,10,18,10,0.0,2,20.0,1
